In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import time

# Suppress TensorFlow warnings for cleaner output
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

print(f"TensorFlow Version: {tf.__version__}")
print(f"NumPy Version: {np.__version__}")

# ==============================================================================
# PART 1: EDGE AI PROTOTYPE (TENSORFLOW LITE)
# Objective: Train a lightweight model, convert it to TFLite for Edge devices,
# and test its performance.
# ==============================================================================

print("\n--- Starting Part 1: Edge AI Prototype ---")

# --- 1.1 Load and Preprocess Data ---

# We use CIFAR-10, a standard dataset.
# For the assignment's "recyclable items" theme, we'll map the classes:
# 'Recyclable' (Man-made): airplane, automobile, ship, truck
# 'Organic' (Natural): bird, cat, deer, dog, frog, horse
# We will create a binary classifier (Recyclable=1, Organic=0)

(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

# CIFAR-10 class names
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# Indices for our mapping
recyclable_indices = [0, 1, 8, 9]  # airplane, automobile, ship, truck
organic_indices = [2, 3, 4, 5, 6, 7] # bird, cat, deer, dog, frog, horse

def preprocess_and_map_labels(images, labels):
    """Maps CIFAR-10 labels to binary 'Recyclable' (1) or 'Organic' (0)"""
    # Normalize images from [0, 255] to [0, 1]
    images = images.astype('float32') / 255.0

    # Flatten label array
    labels = labels.flatten()

    # Create new binary labels
    # Initialize with -1 to catch errors
    new_labels = np.full(labels.shape, -1, dtype=int)

    for idx, label in enumerate(labels):
        if label in recyclable_indices:
            new_labels[idx] = 1  # Recyclable
        elif label in organic_indices:
            new_labels[idx] = 0  # Organic

    # Find images that belong to our target classes
    mask = new_labels != -1
    filtered_images = images[mask]
    filtered_labels = new_labels[mask]

    # Add a channel dimension for Keras (e.g., (32, 32, 3))
    # CIFAR-10 is already in this format, so this step is just a confirmation.

    return filtered_images, filtered_labels.reshape(-1, 1) # Reshape for Keras

x_train, y_train = preprocess_and_map_labels(x_train_full, y_train_full)
x_test, y_test = preprocess_and_map_labels(x_test_full, y_test_full)

print(f"Original training images: {len(x_train_full)}, Filtered: {len(x_train)}")
print(f"Original test images: {len(x_test_full)}, Filtered: {len(x_test)}")
print(f"Image shape: {x_train.shape[1:]}")
print(f"Sample labels (0=Organic, 1=Recyclable): {y_train[:10].flatten()}")

# --- 1.2 Build a Lightweight CNN Model ---

def create_lightweight_cnn(input_shape):
    """
    Builds a simple, lightweight CNN suitable for edge devices.
    We keep the number of filters and layers low to ensure a small model size
    and fast inference.
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),

        # Convolutional Block 1
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        # Convolutional Block 2
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        # Flatten and Dense Layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3), # Dropout to prevent overfitting

        # Output Layer (Binary Classification)
        tf.keras.layers.Dense(1, activation='sigmoid') # Sigmoid for 0 or 1
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_lightweight_cnn(x_train.shape[1:])
model.summary()

# --- 1.3 Train the Model ---

print("\nTraining the Keras model...")
# Using a small number of epochs as requested
EPOCHS = 5
BATCH_SIZE = 64

history = model.fit(
    x_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    verbose=1
)

# --- 1.4 Plot Training History ---

def plot_history(history):
    plt.figure(figsize=(12, 4))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(history)

# --- 1.5 Convert Model to TensorFlow Lite (TFLite) ---

print("\nConverting model to TensorFlow Lite...")

# A. Standard Conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# B. Conversion with Post-Training Quantization
# This is crucial for Edge AI. It reduces model size (up to 4x) and
# speeds up inference with minimal accuracy loss by converting
# 32-bit floats to 8-bit integers.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# --- 1.6 Save Models and Compare Size ---

# Create a file path
tflite_model_file = pathlib.Path('recycling_model.tflite')
tflite_quantized_model_file = pathlib.Path('recycling_model_quantized.tflite')

# Save the models
tflite_model_file.write_bytes(tflite_model)
tflite_quantized_model_file.write_bytes(tflite_quantized_model)

# Compare file sizes
keras_model_size = model.count_params() * 4 # Approx size in bytes (float32)
tflite_size = tflite_model_file.stat().st_size
tflite_quantized_size = tflite_quantized_model_file.stat().st_size

print(f"\nKeras Model (approx.): {keras_model_size / 1024:.2f} KB")
print(f"Standard TFLite Model: {tflite_size / 1024:.2f} KB")
print(f"Quantized TFLite Model: {tflite_quantized_size / 1024:.2f} KB")
print(f"Quantization saved {1 - (tflite_quantized_size / tflite_size):.1%} of space!")

# --- 1.7 Run Inference with the TFLite Model ---

print("\nRunning inference with the quantized TFLite model...")

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=str(tflite_quantized_model_file))
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a test image
test_image = np.expand_dims(x_test[0], axis=0).astype(np.float32) # TFLite model expects float32
true_label = y_test[0][0]

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], test_image)

# Run inference
start_time = time.time()
interpreter.invoke()
inference_time = (time.time() - start_time) * 1000 # in ms

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
prediction = output_data[0][0]

# Interpret the result
label_map = {0: 'Organic', 1: 'Recyclable'}
predicted_label_index = 1 if prediction > 0.5 else 0
predicted_label_name = label_map[predicted_label_index]
true_label_name = label_map[true_label]

print(f"\n--- TFLite Inference Result ---")
print(f"Test Image True Label: {true_label_name} (Raw: {true_label})")
print(f"Model Output (Sigmoid): {prediction:.4f}")
print(f"Predicted Label: {predicted_label_name}")
print(f"Inference Time: {inference_time:.2f} ms")
print("---------------------------------")
print("This very fast inference time is what allows Edge AI to power")
print("real-time applications like autonomous drones or live translation.")


# ==============================================================================
# PART 2: AI-DRIVEN IOT CONCEPT (SMART AGRICULTURE SIMULATION)
# Objective: Simulate sensor data and use a simple AI model to predict
# the need for irrigation.
# ==============================================================================

print("\n--- Starting Part 2: AI-Driven IoT Simulation ---")

class SmartAgriSystem:
    """
    A simulation of a Smart Agriculture system using AI and IoT.
    """
    def __init__(self):
        self.hour = 0
        self.sensor_data = self._generate_dummy_data()
        print("Smart Agriculture System Initialized.")
        print("Sensors: Soil Moisture (%), Temperature (C), Humidity (%)")

    def _generate_dummy_data(self):
        """Generates 24 hours of plausible sensor data."""
        hours = np.arange(0, 24)

        # Simulate Temperature: cooler at night, warmer at day
        temp_base = 15 + 10 * np.sin(np.pi * (hours - 6) / 12)
        temp = temp_base + np.random.normal(0, 0.5, 24) # Add noise

        # Simulate Humidity: inverse of temperature
        humidity_base = 60 - 20 * np.sin(np.pi * (hours - 6) / 12)
        humidity = humidity_base + np.random.normal(0, 2, 24)
        humidity = np.clip(humidity, 0, 100) # Clamp values

        # Simulate Soil Moisture: starts high, slowly decreases (evaporation)
        moisture = 55 - (hours * 1.5) + np.random.normal(0, 1, 24)
        moisture = np.clip(moisture, 0, 100)

        return {
            'temperature': temp,
            'humidity': humidity,
            'soil_moisture': moisture
        }

    def read_sensor_data(self, hour):
        """Reads the simulated sensor data for a given hour."""
        if not 0 <= hour < 24:
            raise ValueError("Hour must be between 0 and 23")

        data = {
            'temp': self.sensor_data['temperature'][hour],
            'humid': self.sensor_data['humidity'][hour],
            'moisture': self.sensor_data['soil_moisture'][hour]
        }
        return data

    def predict_irrigation(self, temp, humid, moisture):
        """
        This is the 'AI Model'. In a real system, this would be a trained
        ML model. For this simulation, we use a rule-based expert system.

        Prediction: Predicted crop yield (as a concept) is low if irrigation
        is not applied when needed.
        """
        # Rule 1: Soil is too dry.
        if moisture < 30:
            # Rule 2: It's hot, so water will evaporate (evapotranspiration)
            if temp > 25:
                return "IRRIGATION NEEDED (High Evaporation Risk)"
            else:
                return "IRRIGATION NEEDED (Low Soil Moisture)"

        # Rule 3: Soil is moist, but it's extremely hot
        if moisture < 45 and temp > 32:
            return "IRRIGATION NEEDED (Heat Stress Risk)"

        # Rule 4: Conditions are fine
        return "No Irrigation Needed"

    def run_simulation(self):
        """Runs the 24-hour simulation loop."""
        print("\n--- Running 24-Hour Smart Farm Simulation ---\n")
        print("Hour | Temp (C) | Humid (%) | Soil (%) | AI Decision")
        print("-" * 60)

        for hour in range(24):
            # 1. IoT Sensors collect data
            data = self.read_sensor_data(hour)

            # 2. AI Model processes data (on the Edge or Cloud)
            decision = self.predict_irrigation(
                data['temp'], data['humid'], data['moisture']
            )

            # 3. Data Flow: Print results
            print(f" {hour:02d}:00 |  {data['temp']:>6.1f}  |  {data['humid']:>7.1f}  |  {data['moisture']:>6.1f}  | {decision}")

            # 4. Action: If irrigation is needed, we'd trigger an actuator.
            if "IRRIGATION NEEDED" in decision:
                # In a real system, this would happen:
                # self.trigger_actuator('water_pump', 'ON')

                # We can simulate the moisture increase
                if hour + 1 < 24:
                    self.sensor_data['soil_moisture'][hour+1:] += 20
                    self.sensor_data['soil_moisture'] = np.clip(
                        self.sensor_data['soil_moisture'], 0, 95
                    )
                    print(f"      -> ACTION: Water pump activated. Soil moisture increased.")

# --- Run the IoT Simulation ---
iot_system = SmartAgriSystem()
iot_system.run_simulation()

print("\n--- All Tasks Completed Successfully ---")